In [1]:
from sentence_transformers import SentenceTransformer
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'

model = SentenceTransformer('all-MiniLM-L6-v2', device=device)
model

SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
)

In [4]:
from elasticsearch import Elasticsearch

ELASTCSEARCH_CERT_PATH = "/Users/liuxg/elastic/elasticsearch-8.10.0/config/certs/http_ca.crt"
 
client = Elasticsearch(  ['https://localhost:9200'],
    basic_auth = ('elastic', 'vXDWYtL*my3vnKY9zCfL'),
    ca_certs = ELASTCSEARCH_CERT_PATH,
    verify_certs = True)
print(client.info())

{'name': 'liuxgm.local', 'cluster_name': 'elasticsearch', 'cluster_uuid': 'lM5ZgEQNTkO8phtT5qZAHQ', 'version': {'number': '8.10.0', 'build_flavor': 'default', 'build_type': 'tar', 'build_hash': 'e338da74c79465dfdc204971e600342b0aa87b6b', 'build_date': '2023-09-07T08:16:21.960703010Z', 'build_snapshot': False, 'lucene_version': '9.7.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'}


In [12]:
INDEX_NAME = "book_index"

if es.indices.exists(index=INDEX_NAME):
    print("Deleting existing %s" % INDEX_NAME)
    client.options(ignore_status=[400, 404]).indices.delete(index=INDEX_NAME)

# Define the mapping
mappings = {
    "properties": {
        "title_vector": {
            "type": "dense_vector",
            "dims": 384,
            "index": "true",
            "similarity": "cosine"
        }
    }
}


# Create the index
client.indices.create(index = INDEX_NAME, mappings = mappings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'book_index'})

In [13]:
import json

# Load data into a JSON object
with open('books.json') as f:
   data_json = json.load(f)
 
actions = []
for book in data_json:
    actions.append({"index": {"_index": INDEX_NAME}})
    # Transforming the title into an embedding using the model
    book["title_vector"] = model.encode(book["title"]).tolist()
    actions.append(book)
client.bulk(index=INDEX_NAME, operations=actions)


ObjectApiResponse({'errors': False, 'took': 55, 'items': [{'index': {'_index': 'book_index', '_id': 'cL8-DYsByaLf0EuT882D', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 0, '_primary_term': 1, 'status': 201}}, {'index': {'_index': 'book_index', '_id': 'cb8-DYsByaLf0EuT882D', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 1, '_primary_term': 1, 'status': 201}}, {'index': {'_index': 'book_index', '_id': 'cr8-DYsByaLf0EuT882D', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 2, '_primary_term': 1, 'status': 201}}, {'index': {'_index': 'book_index', '_id': 'c78-DYsByaLf0EuT882D', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 3, '_primary_term': 1, 'status': 201}}, {'index': {'_index': 'book_index', '_id': 'dL8-DYsByaLf0EuT882D', '_version': 1, 'result': 'created', '_shards

In [14]:
def pretty_response(response):
    for hit in response['hits']['hits']:
        id = hit['_id']
        publication_date = hit['_source']['publish_date']
        score = hit['_score']
        title = hit['_source']['title']
        summary = hit['_source']['summary']
        publisher = hit["_source"]["publisher"]
        num_reviews = hit["_source"]["num_reviews"]
        authors = hit["_source"]["authors"]
        pretty_output = (f"\nID: {id}\nPublication date: {publication_date}\nTitle: {title}\nSummary: {summary}\nPublisher: {publisher}\nReviews: {num_reviews}\nAuthors: {authors}\nScore: {score}")
        print(pretty_output)

In [15]:
response = client.search(index="book_index", body={
    "knn": {
      "field": "title_vector",
      "query_vector": model.encode("Best javascript books?"),
      "k": 10,
      "num_candidates": 100
    }
})

pretty_response(response)


ID: eL8-DYsByaLf0EuT882D
Publication date: 2008-05-15
Title: JavaScript: The Good Parts
Summary: A deep dive into the parts of JavaScript that are essential to writing maintainable code
Publisher: oreilly
Reviews: 51
Authors: ['douglas crockford']
Score: 0.80752474

ID: dL8-DYsByaLf0EuT882D
Publication date: 2015-03-27
Title: You Don't Know JS: Up & Going
Summary: Introduction to JavaScript and programming as a whole
Publisher: oreilly
Reviews: 36
Authors: ['kyle simpson']
Score: 0.6946183

ID: db8-DYsByaLf0EuT882D
Publication date: 2018-12-04
Title: Eloquent JavaScript
Summary: A modern introduction to programming
Publisher: no starch press
Reviews: 38
Authors: ['marijn haverbeke']
Score: 0.6617909

ID: cL8-DYsByaLf0EuT882D
Publication date: 2019-10-29
Title: The Pragmatic Programmer: Your Journey to Mastery
Summary: A guide to pragmatic programming for software engineers and developers
Publisher: addison-wesley
Reviews: 30
Authors: ['andrew hunt', 'david thomas']
Score: 0.61159486



/var/folders/kw/pvmj1zgs44l8t32cs8blf6sc0000gn/T/ipykernel_23541/2388497951.py:1: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  response = client.search(index="book_index", body={


In [25]:
response = client.search(index=INDEX_NAME, body={
    "knn": {
      "field": "title_vector",
      "query_vector": model.encode("Best javascript books?"),
      "k": 10,
      "num_candidates": 100,
      "filter": {
          "term": {
              "publisher.keyword": "addison-wesley"
          }
      }
    }
})

pretty_response(response)


ID: cL8-DYsByaLf0EuT882D
Publication date: 2019-10-29
Title: The Pragmatic Programmer: Your Journey to Mastery
Summary: A guide to pragmatic programming for software engineers and developers
Publisher: addison-wesley
Reviews: 30
Authors: ['andrew hunt', 'david thomas']
Score: 0.61159486

ID: dr8-DYsByaLf0EuT882D
Publication date: 1994-10-31
Title: Design Patterns: Elements of Reusable Object-Oriented Software
Summary: Guide to design patterns that can be used in any object-oriented language
Publisher: addison-wesley
Reviews: 45
Authors: ['erich gamma', 'richard helm', 'ralph johnson', 'john vlissides']
Score: 0.5617569


/var/folders/kw/pvmj1zgs44l8t32cs8blf6sc0000gn/T/ipykernel_23541/56853983.py:1: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  response = client.search(index=INDEX_NAME, body={


In [27]:
response = client.search(index="book_index", body={
    "knn": {
      "field": "title_vector",
      "query_vector": model.encode("Best javascript books?"),
      "k": 10,
      "num_candidates": 100,
      "filter": {
          "bool": {
              "should": [
                  {
                    "term": {
                        "publisher.keyword": "addison-wesley"
                    }
                  },
                  {
                    "term": {
                        "authors.keyword": "robert c. martin"
                    }
                  }
              ],

          }
      }
    }
})

pretty_response(response)


ID: cL8-DYsByaLf0EuT882D
Publication date: 2019-10-29
Title: The Pragmatic Programmer: Your Journey to Mastery
Summary: A guide to pragmatic programming for software engineers and developers
Publisher: addison-wesley
Reviews: 30
Authors: ['andrew hunt', 'david thomas']
Score: 0.61159486

ID: d78-DYsByaLf0EuT882D
Publication date: 2011-05-13
Title: The Clean Coder: A Code of Conduct for Professional Programmers
Summary: A guide to professional conduct in the field of software engineering
Publisher: prentice hall
Reviews: 20
Authors: ['robert c. martin']
Score: 0.57042736

ID: dr8-DYsByaLf0EuT882D
Publication date: 1994-10-31
Title: Design Patterns: Elements of Reusable Object-Oriented Software
Summary: Guide to design patterns that can be used in any object-oriented language
Publisher: addison-wesley
Reviews: 45
Authors: ['erich gamma', 'richard helm', 'ralph johnson', 'john vlissides']
Score: 0.5617569

ID: c78-DYsByaLf0EuT882D
Publication date: 2008-08-11
Title: Clean Code: A Handboo

/var/folders/kw/pvmj1zgs44l8t32cs8blf6sc0000gn/T/ipykernel_23541/1445678156.py:1: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  response = client.search(index="book_index", body={


In [23]:
response

ObjectApiResponse({'took': 0, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 0, 'relation': 'eq'}, 'max_score': None, 'hits': []}})

In [28]:
response = client.search(index="book_index", body={
    "query": {
        "match": {
            "summary": "python"
        }
    },
    "knn": {
        "field": "title_vector",
        # generate embedding for query so it can be compared to `title_vector`
        "query_vector" : model.encode("python programming").tolist(),
        "k": 5,
        "num_candidates": 10
    },
    "rank": {
        "rrf": {
            "window_size": 100,
            "rank_constant": 20
        }
    }
})

pretty_response(response)


ID: cb8-DYsByaLf0EuT882D
Publication date: 2019-05-03
Title: Python Crash Course
Summary: A fast-paced, no-nonsense guide to programming in Python
Publisher: no starch press
Reviews: 42
Authors: ['eric matthes']
Score: None

ID: cL8-DYsByaLf0EuT882D
Publication date: 2019-10-29
Title: The Pragmatic Programmer: Your Journey to Mastery
Summary: A guide to pragmatic programming for software engineers and developers
Publisher: addison-wesley
Reviews: 30
Authors: ['andrew hunt', 'david thomas']
Score: None

ID: eb8-DYsByaLf0EuT882D
Publication date: 2012-06-27
Title: Introduction to the Theory of Computation
Summary: Introduction to the theory of computation and complexity theory
Publisher: cengage learning
Reviews: 33
Authors: ['michael sipser']
Score: None

ID: d78-DYsByaLf0EuT882D
Publication date: 2011-05-13
Title: The Clean Coder: A Code of Conduct for Professional Programmers
Summary: A guide to professional conduct in the field of software engineering
Publisher: prentice hall
Review

/var/folders/kw/pvmj1zgs44l8t32cs8blf6sc0000gn/T/ipykernel_23541/2859249433.py:1: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  response = client.search(index="book_index", body={
